In [53]:
import tensorflow as tf
import numpy as np
import sagemaker
import boto3
import os
import MODEL  # MODEL.py must be download from Codecommit

class FLClient(object):
    def __init__(self, member_ID, local_epochs, x_train_client, y_train_client, x_test_client, y_test_client):
                
        # record the latest processed training round
        self.num_train_per_client = 0
        self.num_test_per_client = 0

        # set client ID
        self.nodeId = member_ID #client member Id

        # set local epochs
        self.epochs_client = local_epochs #the epoch number for local training
                
        # get client local data
        self.x_train_client = np.load(x_train_client)
        self.y_train_client = np.load(y_train_client)
        self.x_test_client = np.load(x_test_client)
        self.y_test_client = np.load(y_test_client)
        
        self.num_train_per_client = self.x_train_client.shape[0]
        self.num_test_per_client = self.x_test_client.shape[0]
        print("Local datasets:")
        print("x_train =", self.x_train_client.shape)
        print("y_train =", self.y_train_client.shape)
        print("x_test =", self.x_test_client.shape)
        print("y_test =", self.y_test_client.shape)  
        print()
        
    # local training at the client with given client train/test data, and files for initial model
    def localTraining (self, round_id, global_model_file):
            
        # load the model received from server an training
        if (global_model_file == 'NA'): #no weights received from server, only the model configuration file ".py"
            # check accuracy and loss of the initial model at round 0
            print("Client " + str(self.nodeId) + ": ")
            print("2: Download a global model")
            mlmodel = MODEL.MLMODEL()
            model = mlmodel.getModel()
            prio_train_loss, prio_train_acc = "NA", "NA"
            prio_test_loss, prio_test_acc   = "NA", "NA"
            print("prio local training: training loss: {} \t training accuracy: {}".format(prio_train_loss,prio_train_acc) )
            print("prio local training: testing loss: {} \t testing accuracy: {}".format(prio_test_loss,prio_test_acc))
            print()
            
            print("3: Local training ...")
            model.fit(self.x_train_client, self.y_train_client, epochs=self.epochs_client)
            after_train_loss, after_train_acc = model.evaluate(self.x_train_client,  self.y_train_client, verbose=2)
            after_test_loss, after_test_acc  = model.evaluate(self.x_test_client,  self.y_test_client, verbose=2)    
            print('after local training: training loss: %2.4f \t training accuracy: %2.4f' % (after_train_loss, after_train_acc))
            print('after local training: testing loss: %2.4f \t testing accuracy: %2.4f \n' % (after_test_loss, after_test_acc))
            
        else:
            # download the model received from server s3_address and global_model_file
            mlmodel = MODEL.MLMODEL()
            model = mlmodel.getModel()
            print("Client " + str(self.nodeId) + ": ")
            print("2: Download a global model")
            weights_prio_training= np.load("models/" + global_model_file, allow_pickle=True) 
            model.set_weights(weights_prio_training)
        
            # check local accuracy and loss
            prio_train_loss, prio_train_acc = model.evaluate(self.x_train_client,  self.y_train_client, verbose=2)
            prio_test_loss, prio_test_acc  = model.evaluate(self.x_test_client,  self.y_test_client, verbose=2)
            print('prio local training: training loss: %2.4f \t training accuracy: %2.4f' % (prio_train_loss, prio_train_acc))
            print('prio local training: testing loss: %2.4f \t testing accuracy: %2.4f' % (prio_test_loss, prio_test_acc))
            print()
            
            print("3: Local training ...")
            # train based on the avg model from server
            model.fit(self.x_train_client,self.y_train_client, epochs=self.epochs_client)
            after_train_loss, after_train_acc = model.evaluate(self.x_train_client,  self.y_train_client, verbose=2)
            after_test_loss, after_test_acc  = model.evaluate(self.x_test_client,  self.y_test_client, verbose=2)
            print('after local training: training loss: %2.4f \t training accuracy: %2.4f' % (after_train_loss, after_train_acc))
            print('after local training: testing loss: %2.4f \t testing accuracy: %2.4f \n' % (after_test_loss, after_test_acc))
            
        # save weights at file server
        savedModelFileName = 'train_weight_round_{}_client_{}.npy'.format(round_id, self.nodeId)
        weights = model.get_weights()
        np.save("models/" + savedModelFileName, weights)
    
#         return savedModelFileName
        return savedModelFileName, self.num_train_per_client, after_train_loss, after_train_acc, after_test_loss, after_test_acc

In [54]:
sess = sagemaker.Session()
bucket = sess.default_bucket() 
s3 = boto3.resource('s3') 

# init clients
numClients = 2
local_epochs = 5
clients = []
for i in range(1, numClients+1, 1): 
    prefix = 'local_dataset'
    
    key = 'x_train_client_' + str(i) + '.npy'
#     s3.Bucket(bucket).download_file(os.path.join(prefix, key), key)
    x_train_client = key
    
    key = 'y_train_client_' + str(i) + '.npy'
#     s3.Bucket(bucket).download_file(os.path.join(prefix, key), key)
    y_train_client = key
    
    key = 'x_test_client_' + str(i) + '.npy'
#     s3.Bucket(bucket).download_file(os.path.join(prefix, key), key)
    x_test_client = key
    
    key = 'y_test_client_' + str(i) + '.npy'
#     s3.Bucket(bucket).download_file(os.path.join(prefix, key), key)
    y_test_client = key

    client = FLClient(i, local_epochs, x_train_client, y_train_client, x_test_client, y_test_client)  
    clients.append(client)


Local datasets:
x_train = (20000, 28, 28)
y_train = (20000,)
x_test = (3000, 28, 28)
y_test = (3000,)

Local datasets:
x_train = (20000, 28, 28)
y_train = (20000,)
x_test = (3000, 28, 28)
y_test = (3000,)



In [67]:
def weightedMeanSequence(matrixSeq, weights):
        assert len(matrixSeq) == len(weights)
        total_weight = 0.0
        base = [0]*matrixSeq[0] #initialize
        for w in range(len(matrixSeq)):  # w is the number of local samples
            total_weight += weights[w]
            base = base + matrixSeq[w]*weights[w] 
        weighted_matrix = [v / total_weight for v in base]
        return weighted_matrix
    
numRounds = 2
roundId = 0 
model_params_w = []
global_model_file = 'NA'
numClientSamples = []
train_loss_params = []
train_acc_params = []
test_loss_params = []
test_acc_params = []

while(roundId < numRounds): 
    # Local training
    for c in clients:
        savedModelFileName, num_train_per_client, after_train_loss, after_train_acc, after_test_loss, after_test_acc = c.localTraining(roundId, global_model_file)
        model_weights = np.load("models/" + savedModelFileName, allow_pickle=True)
        model_params_w.append(model_weights)
        
        numClientSamples.append(num_train_per_client)
        train_loss_params.append(np.array(float(after_train_loss)))
        train_acc_params.append(np.array(float(after_train_acc)))
        test_loss_params.append(np.array(float(after_test_loss)))
        test_acc_params.append(np.array(float(after_test_acc)))

    # weighted_avg
    avg_model_params_w = weightedMeanSequence(model_params_w, numClientSamples)
    avg_loss = weightedMeanSequence(train_loss_params, numClientSamples)
    avg_acc = weightedMeanSequence(train_acc_params, numClientSamples)
    print("Server: Round: " + str(roundId) + ", loss: " + str(avg_loss[0]) + ", accuracy: " + str(avg_acc[0]) + "\n") 

    roundId = roundId + 1
    global_model_file = 'train_weight_round_' + str(roundId) + '.npy'
    file_name = "models/" + global_model_file
    np.save(file_name, avg_model_params_w) 


Client 1: 
2: Download a global model
prio local training: training loss: NA 	 training accuracy: NA
prio local training: testing loss: NA 	 testing accuracy: NA

3: Local training ...
Epoch 1/5
625/625 [==============================] - 2s 3ms/step - loss: 0.4444 - accuracy: 0.8734
Epoch 2/5
625/625 [==============================] - 1s 2ms/step - loss: 0.2201 - accuracy: 0.9380
Epoch 3/5
625/625 [==============================] - 2s 3ms/step - loss: 0.1620 - accuracy: 0.9518
Epoch 4/5
625/625 [==============================] - 2s 2ms/step - loss: 0.1264 - accuracy: 0.9621
Epoch 5/5
625/625 [==============================] - 1s 2ms/step - loss: 0.1020 - accuracy: 0.9695
625/625 - 1s - loss: 0.0596 - accuracy: 0.9836
94/94 - 0s - loss: 0.1561 - accuracy: 0.9553
after local training: training loss: 0.0596 	 training accuracy: 0.9836
after local training: testing loss: 0.1561 	 testing accuracy: 0.9553 

Client 2: 
2: Download a global model
prio local training: training loss: NA 	 train